In [88]:
import arcpy
import os
import glob
import numpy as np
import pandas as pd

In [89]:
in_excel = "warnings_balkan copy.xlsx"

In [91]:
fgdb = arcpy.CreateFileGDB_management(out_folder_path=os.getcwd(), out_name="Workspace.gdb").getOutput(0)

In [92]:
fgdb_table = arcpy.ExcelToTable_conversion(in_excel, os.path.join(fgdb, "warnings_balkan")).getOutput(0)

In [93]:
input_schema_dict = {}
for field in arcpy.ListFields(fgdb_table):
    input_schema_dict[field.name] = field.type

In [94]:
input_schema_dict

{'AWTCaption': 'String',
 'AreaCaption': 'String',
 'CountryCaption': 'String',
 'EMMA_ID': 'String',
 'OBJECTID': 'OID',
 'Polygon': 'String',
 'WaFrom': 'Date',
 'WaId': 'Integer',
 'WaLevel': 'Integer',
 'WaText': 'String',
 'WaTo': 'Date',
 'WaType': 'Integer'}

In [95]:
# input_fields = [field.name for field in arcpy.ListFields(fgdb_table)]
input_fields = list(input_schema_dict.keys())

In [96]:
poly_fc = arcpy.CreateFeatureclass_management(out_path=fgdb, out_name="warnings_balkan_poly", geometry_type="POLYGON", spatial_reference="4326").getOutput(0)

In [97]:
# Add needed attribute fields to the target poly FC
for field in input_schema_dict:
    if field == "OBJECTID":
        pass
    else:
        print("Adding field {0}...".format(field))
        # Add field with data type from schema dict
        arcpy.AddField_management(poly_fc, str(field), input_schema_dict[field])

Adding field WaId...
Adding field CountryCaption...
Adding field AreaCaption...
Adding field EMMA_ID...
Adding field Polygon...
Adding field WaLevel...
Adding field AWTCaption...
Adding field WaType...
Adding field WaFrom...
Adding field WaTo...
Adding field WaText...


In [98]:
# Helper function to create polygon object for an insert cursor
def prepare_polygon_object(poly_coords):
    single_feature_geometry = []
    for coord_pair in poly_coords.split(" "):
        if coord_pair == '':
            pass
        else:
            # Change string to floats
            float_coord_pair = [float(coord) for coord in coord_pair.split(",")]
            single_feature_geometry.append(float_coord_pair)
    all_features_geometry = [single_feature_geometry]
    
    polygon_objects_list = []

    for feature in all_features_geometry:
        # Create a point object
        list_of_points = []
        for coordinate_pair in feature:
            y_vertex = coordinate_pair[0]
            x_vertex = coordinate_pair[1]
            point_object = arcpy.Point(x_vertex, y_vertex)
            list_of_points.append(point_object)

        # Create an arcpy array
        ap_array = arcpy.Array(list_of_points)
        # Create a polygon object
        polygon_obj = arcpy.Polygon(ap_array)
        # Append to the objects list
        polygon_objects_list.append(polygon_obj)
    
    return polygon_objects_list

In [99]:
insertcursor_field_list = ["SHAPE@"] + list(input_schema_dict.keys())

['SHAPE@',
 'OBJECTID',
 'WaId',
 'CountryCaption',
 'AreaCaption',
 'EMMA_ID',
 'Polygon',
 'WaLevel',
 'AWTCaption',
 'WaType',
 'WaFrom',
 'WaTo',
 'WaText']

In [69]:
# # Start iterating on the target Polygonal FC with an insert cursor
# with arcpy.da.InsertCursor(poly_fc, insertcursor_field_list) as insertcursor:
    
#     # Start iterating on the input file geodatabase table with a searchcursor
#     with arcpy.da.SearchCursor(fgdb_table, input_fields) as searchcursor:
#         for row in searchcursor:
#             objectid = row[0]
#             waid = row[1]
#             country_caption = row[2]
#             area_caption = row[3]
#             emma_id = row[4]
#             polygon = 0
#             walevel = row[6]
#             awtcaption = row[7]
#             watype = row[8]
#             wafrom = row[9]
#             wato = row[10]
#             watext = row[11]
#             print("Processing row with Object ID of {0}".format(str(objectid)))
            
#             polygon_objects_list = prepare_polygon_object(polygon)
#             insert_vals = (polygon_objects_list, 
#                            objectid,
#                            waid,
#                            country_caption,
#                            area_caption,
#                            emma_id,
#                            walevel,
#                            awtcaption,
#                            watype,
#                            wafrom,
#                            wato,
#                            watext)
#             print(insert_vals)
#             insertcursor.insertRow(insert_vals)
            
#             try:
#                 polygon_objects_list = prepare_polygon_object(polygon)
#                 insert_vals = (polygon_objects_list, 
#                               objectid,
#                               waid,
#                               country_caption,
#                               area_caption,
#                               emma_id,
#                               walevel,
#                               awtcaption,
#                               watype,
#                               wafrom,
#                               wato,
#                               watext)
#                 print(insert_vals)
#                 insertcursor.insertRow(insert_vals)
#             except ValueError:
#                 print("Could not read geometry from input list. Bypassing...")

Processing row with Object ID of 1


AttributeError: 'int' object has no attribute 'split'

In [100]:
# Get fields in the excel
fields = [field.name for field in arcpy.ListFields(fgdb_table)]

# Start iterating on the target Polygonal FC with an insert cursor
with arcpy.da.InsertCursor(poly_fc, ["SHAPE@"]) as insertcursor:
    
    # Start iterating on the input file geodatabase table with a searchcursor
    with arcpy.da.SearchCursor(fgdb_table, fields) as searchcursor:
        for row in searchcursor:
            objectid = row[0]
            waid = row[1]
            country_caption = row[2]
            area_caption = row[3]
            emma_id = row[4]
            polygon = row[5]
            walevel = row[6]
            awtcaption = row[7]
            watype = row[8]
            wafrom = row[9]
            wato = row[10]
            watext = row[11]
            print("Processing row with Object ID of {0}".format(str(objectid)))
            
            try:
                polygon_objects_list = prepare_polygon_object(polygon)
                insertcursor.insertRow(polygon_objects_list)
            except ValueError:
                print("Could not read geometry from input list. Bypassing...")

Processing row with Object ID of 1
Processing row with Object ID of 2
Processing row with Object ID of 3
Processing row with Object ID of 4
Processing row with Object ID of 5
Processing row with Object ID of 6
Processing row with Object ID of 7
Processing row with Object ID of 8
Processing row with Object ID of 9
Processing row with Object ID of 10
Processing row with Object ID of 11
Processing row with Object ID of 12
Processing row with Object ID of 13
Processing row with Object ID of 14
Processing row with Object ID of 15
Processing row with Object ID of 16
Processing row with Object ID of 17
Processing row with Object ID of 18
Processing row with Object ID of 19
Processing row with Object ID of 20
Processing row with Object ID of 21
Processing row with Object ID of 22
Processing row with Object ID of 23
Processing row with Object ID of 24
Processing row with Object ID of 25
Processing row with Object ID of 26
Processing row with Object ID of 27
Processing row with Object ID of 28
P

Processing row with Object ID of 242
Processing row with Object ID of 243
Processing row with Object ID of 244
Processing row with Object ID of 245
Processing row with Object ID of 246
Processing row with Object ID of 247
Processing row with Object ID of 248
Processing row with Object ID of 249
Processing row with Object ID of 250
Processing row with Object ID of 251
Processing row with Object ID of 252
Processing row with Object ID of 253
Processing row with Object ID of 254
Processing row with Object ID of 255
Processing row with Object ID of 256
Processing row with Object ID of 257
Processing row with Object ID of 258
Processing row with Object ID of 259
Processing row with Object ID of 260
Processing row with Object ID of 261
Processing row with Object ID of 262
Processing row with Object ID of 263
Processing row with Object ID of 264
Processing row with Object ID of 265
Processing row with Object ID of 266
Processing row with Object ID of 267
Processing row with Object ID of 268
P

Processing row with Object ID of 671
Processing row with Object ID of 672
Processing row with Object ID of 673
Processing row with Object ID of 674
Processing row with Object ID of 675
Processing row with Object ID of 676
Processing row with Object ID of 677
Processing row with Object ID of 678
Processing row with Object ID of 679
Processing row with Object ID of 680
Processing row with Object ID of 681
Processing row with Object ID of 682
Processing row with Object ID of 683
Processing row with Object ID of 684
Processing row with Object ID of 685
Processing row with Object ID of 686
Processing row with Object ID of 687
Processing row with Object ID of 688
Processing row with Object ID of 689
Processing row with Object ID of 690
Processing row with Object ID of 691
Processing row with Object ID of 692
Processing row with Object ID of 693
Processing row with Object ID of 694
Processing row with Object ID of 695
Processing row with Object ID of 696
Processing row with Object ID of 697
P

Processing row with Object ID of 1297
Processing row with Object ID of 1298
Processing row with Object ID of 1299
Processing row with Object ID of 1300
Processing row with Object ID of 1301
Processing row with Object ID of 1302
Processing row with Object ID of 1303
Processing row with Object ID of 1304
Processing row with Object ID of 1305
Processing row with Object ID of 1306
Processing row with Object ID of 1307
Processing row with Object ID of 1308
Processing row with Object ID of 1309
Processing row with Object ID of 1310
Processing row with Object ID of 1311
Processing row with Object ID of 1312
Processing row with Object ID of 1313
Processing row with Object ID of 1314
Processing row with Object ID of 1315
Processing row with Object ID of 1316
Processing row with Object ID of 1317
Processing row with Object ID of 1318
Processing row with Object ID of 1319
Processing row with Object ID of 1320
Processing row with Object ID of 1321
Processing row with Object ID of 1322
Processing r

Processing row with Object ID of 1875
Processing row with Object ID of 1876
Processing row with Object ID of 1877
Processing row with Object ID of 1878
Processing row with Object ID of 1879
Processing row with Object ID of 1880
Processing row with Object ID of 1881
Processing row with Object ID of 1882
Processing row with Object ID of 1883
Processing row with Object ID of 1884
Processing row with Object ID of 1885
Processing row with Object ID of 1886
Processing row with Object ID of 1887
Processing row with Object ID of 1888
Processing row with Object ID of 1889
Processing row with Object ID of 1890
Processing row with Object ID of 1891
Processing row with Object ID of 1892
Processing row with Object ID of 1893
Processing row with Object ID of 1894
Processing row with Object ID of 1895
Processing row with Object ID of 1896
Processing row with Object ID of 1897
Processing row with Object ID of 1898
Processing row with Object ID of 1899
Processing row with Object ID of 1900
Processing r

Processing row with Object ID of 2565
Processing row with Object ID of 2566
Processing row with Object ID of 2567
Processing row with Object ID of 2568
Processing row with Object ID of 2569
Processing row with Object ID of 2570
Processing row with Object ID of 2571
Processing row with Object ID of 2572
Processing row with Object ID of 2573
Processing row with Object ID of 2574
Processing row with Object ID of 2575
Processing row with Object ID of 2576
Processing row with Object ID of 2577
Processing row with Object ID of 2578
Processing row with Object ID of 2579
Processing row with Object ID of 2580
Processing row with Object ID of 2581
Processing row with Object ID of 2582
Processing row with Object ID of 2583
Processing row with Object ID of 2584
Processing row with Object ID of 2585
Processing row with Object ID of 2586
Processing row with Object ID of 2587
Processing row with Object ID of 2588
Processing row with Object ID of 2589
Processing row with Object ID of 2590
Processing r

Processing row with Object ID of 3104
Processing row with Object ID of 3105
Processing row with Object ID of 3106
Processing row with Object ID of 3107
Processing row with Object ID of 3108
Processing row with Object ID of 3109
Processing row with Object ID of 3110
Processing row with Object ID of 3111
Processing row with Object ID of 3112
Processing row with Object ID of 3113
Processing row with Object ID of 3114
Processing row with Object ID of 3115
Processing row with Object ID of 3116
Processing row with Object ID of 3117
Processing row with Object ID of 3118
Processing row with Object ID of 3119
Processing row with Object ID of 3120
Processing row with Object ID of 3121
Processing row with Object ID of 3122
Processing row with Object ID of 3123
Processing row with Object ID of 3124
Processing row with Object ID of 3125
Processing row with Object ID of 3126
Processing row with Object ID of 3127
Processing row with Object ID of 3128
Processing row with Object ID of 3129
Processing r

Processing row with Object ID of 3650
Processing row with Object ID of 3651
Processing row with Object ID of 3652
Processing row with Object ID of 3653
Processing row with Object ID of 3654
Processing row with Object ID of 3655
Processing row with Object ID of 3656
Processing row with Object ID of 3657
Processing row with Object ID of 3658
Processing row with Object ID of 3659
Processing row with Object ID of 3660
Processing row with Object ID of 3661
Processing row with Object ID of 3662
Processing row with Object ID of 3663
Processing row with Object ID of 3664
Processing row with Object ID of 3665
Processing row with Object ID of 3666
Processing row with Object ID of 3667
Processing row with Object ID of 3668
Processing row with Object ID of 3669
Processing row with Object ID of 3670
Processing row with Object ID of 3671
Processing row with Object ID of 3672
Processing row with Object ID of 3673
Processing row with Object ID of 3674
Processing row with Object ID of 3675
Processing r

Processing row with Object ID of 4203
Processing row with Object ID of 4204
Processing row with Object ID of 4205
Processing row with Object ID of 4206
Processing row with Object ID of 4207
Processing row with Object ID of 4208
Processing row with Object ID of 4209
Processing row with Object ID of 4210
Processing row with Object ID of 4211
Processing row with Object ID of 4212
Processing row with Object ID of 4213
Processing row with Object ID of 4214
Processing row with Object ID of 4215
Processing row with Object ID of 4216
Processing row with Object ID of 4217
Processing row with Object ID of 4218
Processing row with Object ID of 4219
Processing row with Object ID of 4220
Processing row with Object ID of 4221
Processing row with Object ID of 4222
Processing row with Object ID of 4223
Processing row with Object ID of 4224
Processing row with Object ID of 4225
Processing row with Object ID of 4226
Processing row with Object ID of 4227
Processing row with Object ID of 4228
Processing r

KeyboardInterrupt: 